In [1]:
!git clone https://github.com/sn09/ranking.git

Cloning into 'ranking'...
remote: Enumerating objects: 342, done.
remote: Counting objects: 100% (342/342), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 342 (delta 166), reused 286 (delta 119), pack-reused 0 (from 0)
Receiving objects: 100% (342/342), 169.45 KiB | 3.20 MiB/s, done.
Resolving deltas: 100% (166/166), done.


In [ ]:
import sys

import lightgbm as lgb
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier, LGBMRanker
from sklearn.metrics import log_loss, roc_auc_score
from torch import nn

sys.path.append("./ranking/")

from rankfx.dcnv2.model import DCNv2
from rankfx.finalnet.model import FinalNet

In [3]:
base_path = "/kaggle/input/final-movielens-x1"

df_train = pd.read_csv(f"{base_path}/train.csv")
df_test = pd.read_csv(f"{base_path}/test.csv")
df_val = pd.read_csv(f"{base_path}/valid.csv")

df_train.head()

,label,user_id,item_id,tag_id
0,0,84982,58,39525
1,0,87756,8757,22786
2,0,80311,14912,45676
3,1,82036,84509,33556
4,0,66867,57349,15092


In [4]:
for df in [df_train, df_val, df_test]:
    df["user_id"] = df["user_id"].astype("category")
    df["item_id"] = df["item_id"].astype("category")
    df["tag_id"] = df["tag_id"].astype("category")

# LightGBM

## LGBMClassifier

In [5]:
booster_clf = LGBMClassifier(
    objective="binary",
    max_depth=5,
    learning_rate=1e-1,
    n_estimators=2000,
)

In [6]:
feature_names = df_train.columns.difference(["label"]).tolist()

booster_clf = booster_clf.fit(
    X=df_train.drop(columns="label"),
    y=df_train["label"],
    eval_set=(df_val.drop(columns="label"), df_val["label"]),
    feature_name=df_train.drop(columns="label").columns.tolist(),
    categorical_feature=feature_names,
    callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation()],
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 467878, number of negative: 936923
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072819 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43730
[LightGBM] [Info] Number of data points in the train set: 1404801, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333056 -> initscore=-0.694394
[LightGBM] [Info] Start training from score -0.694394
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's binary_logloss: 0.614768
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's binary_

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7b02b2b8a3e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[37]	valid_0's binary_logloss: 0.47932
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[38]	valid_0's binary_logloss: 0.477549
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[39]	valid_0's binary_logloss: 0.475992
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[40]	valid_0's binary_logloss: 0.473971
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[41]	valid_0's binary_logloss: 0.472167
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[42]	valid_0's binary_logloss: 0.47054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[43]	valid_0's binary_logloss: 0.468779
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[44]	valid_0's binary_logloss: 0.467297
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[45]	valid_0's binary_loglos

In [7]:
booster_clf_preds = booster_clf.predict(
    df_test.drop(columns="label"),
    raw_score=True,
)

In [8]:
roc_auc = roc_auc_score(df_test["label"], booster_clf_preds)
logloss = log_loss(df_test["label"], booster_clf_preds)

print("LightGBM Classifier metrics")
print(f"ROC AUC: {roc_auc}, logloss: {logloss}")

LightGBM Classifier metrics
ROC AUC: 0.9387817972801997, logloss: 2.964612512312728


## LGBMRanker

In [9]:
# dont want to recompile lightgbm, so must ensure group has less than 10000 queries

df_train_lgb = (
    df_train
    .groupby("user_id", as_index=False)
    .sample(n=10_000, replace=True)
    .drop_duplicates()
    .sort_values(by="user_id")
)
group = df_train_lgb["user_id"].value_counts(sort=False).values

df_val_lgb = (
    df_val
    .groupby("user_id", as_index=False)
    .sample(n=10_000, replace=True)
    .drop_duplicates()
    .sort_values(by="user_id")
)
group_val = df_val_lgb["user_id"].value_counts(sort=False).values

/tmp/ipykernel_31/1118170982.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("user_id", as_index=False)
/tmp/ipykernel_31/1118170982.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("user_id", as_index=False)


In [10]:
booster_rnk = LGBMRanker(
    objective="lambdarank",
    max_depth=5,
    learning_rate=1e-1,
    n_estimators=2000,
)

In [11]:
feature_names = df_train_lgb.columns.difference(["label"]).tolist()

booster_rnk = booster_rnk.fit(
    X=df_train_lgb.drop(columns="label"),
    y=df_train_lgb["label"],
    group=group,
    eval_set=[(df_val_lgb.drop(columns="label"), df_val_lgb["label"])],
    eval_group=[group_val],
    feature_name=feature_names,
    categorical_feature=feature_names,
    callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation()],
    eval_metric="auc",
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059933 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43399
[LightGBM] [Info] Number of data points in the train set: 1218503, number of used features: 3
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.654493	valid_0's ndcg@1: 0.859303	valid_0's ndcg@2: 0.868233	valid_0's ndcg@3: 0.875496	valid_0's ndcg@4: 0.879837	valid_0's ndcg@5: 0.883231
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.681099	valid_0's ndcg@1: 0.877182	valid_0's ndcg@2: 0.884895	valid_0's ndcg@3: 0.89

In [12]:
booster_rnk_preds = booster_rnk.predict(
    df_test.drop(columns="label"),
    raw_score=True,
)

In [13]:
roc_auc = roc_auc_score(df_test["label"], booster_rnk_preds)
logloss = log_loss(df_test["label"], booster_rnk_preds)

print("LightGBM Ranker metrics")
print(f"ROC AUC: {roc_auc}, logloss: {logloss}")

LightGBM Ranker metrics
ROC AUC: 0.8492354434477607, logloss: 5.666125678303608


# DCNv2

In [14]:
dcnv2_model = DCNv2(
    model_structure="stacked_parallel",
    use_low_rank_mixture=True,
    cross_low_rank_dim=32,
    num_cross_layers=5,
    num_cross_experts=4,
    parallel_hidden_dims=[256, 512, 1024],
    parallel_dropout=0.2,
    parallel_use_batch_norm=True,
    parallel_activation=nn.ReLU,
    stacked_hidden_dims=[256, 512, 1024],
    stacked_dropout=0.2,
    stacked_use_batch_norm=True,
    stacked_activation=nn.ReLU,
    output_dim=1,
    proj_output_embeddings=True,
)

In [15]:
train_metrics_dcnv2, val_metrics_dcnv2 = dcnv2_model.fit(
    features=df_train.drop(columns="label"),
    target=df_train["label"],
    val_features=df_val.drop(columns="label"),
    val_target=df_val["label"],
    optimizer_cls="torch.optim.Adam",
    optimizer_params=dict(lr=1e-3),
    scheduler_cls="torch.optim.lr_scheduler.ReduceLROnPlateau",
    scheduler_params=dict(mode="max", factor=0.1, patience=2, min_lr=1e-6),
    grad_clip_threshold=10.,
    num_epochs=10,
    seed=42,
    artifacts_path="./dcnv2_artifacts",
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
    eval_metric_name="AUC",
    eval_mode="max",
    embedded_features=["user_id", "item_id", "tag_id"],
    default_embedding_size=64,
    oov_masking_proba=0.05,
    l2_net_reg=0,
    l2_embedding_reg=0,
)

[2025-05-03 22:21:44,116]{model.py:660} - INFO - Used features config: FeaturesConfig(features=[Feature(name='user_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=64, embedding_vocab_size=16976, embedding_padding_idx=None), Feature(name='item_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=64, embedding_vocab_size=23605, embedding_padding_idx=None), Feature(name='tag_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=64, embedding_vocab_size=49658, embedding_padding_idx=None)])
[2025-05-03 22:21:47,298]{model.py:675} - INFO - Artifacts path is /kaggle/working/dcnv2_artifacts
[2025-05-03 22:21:47,299]{model.py:677} - INFO - Creating artifacts path
[2025-05-03 22:21:47,300]{model.py:681} - INFO - Best model path is dcnv2_artifacts/best_model.pt
[2025-05-03 22:21:47,616]{model.py:686} - INFO - Building features 

Train epoch #0:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 22:23:05,842]{model.py:567} - INFO - Finished Train Epoch #0, average metrics - [loss: 0.47479]


Validation epoch #0:   0%|          | 0/98 [00:00<?, ?it/s]

[2025-05-03 22:23:15,082]{model.py:567} - INFO - Finished Validation Epoch #0, average metrics - [AUC: 0.88930, log_loss: 0.34868]
[2025-05-03 22:23:25,561]{model.py:747} - INFO - Best model with AUC = 0.8893002081617837 was saved to dcnv2_artifacts/best_model.pt


...

[2025-05-03 22:37:34,274]{model.py:567} - INFO - Finished Validation Epoch #8, average metrics - [AUC: 0.93857, log_loss: 0.30407]
[2025-05-03 22:37:44,776]{model.py:747} - INFO - Best model with AUC = 0.9385710490776249 was saved to dcnv2_artifacts/best_model.pt


Train epoch #9:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 22:39:03,031]{model.py:567} - INFO - Finished Train Epoch #9, average metrics - [loss: 0.17956]


Validation epoch #9:   0%|          | 0/98 [00:00<?, ?it/s]

[2025-05-03 22:39:22,500]{model.py:567} - INFO - Finished Validation Epoch #9, average metrics - [AUC: 0.93843, log_loss: 0.31329]
[2025-05-03 22:39:32,907]{model.py:763} - INFO - Loading best model from dcnv2_artifacts/best_model.pt


In [16]:
test_metrics_dcnv2 = dcnv2_model.test(
    features=df_test.drop(columns="label"),
    target=df_test["label"],
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
)
test_metrics_dcnv2

[2025-05-03 22:39:32,988]{model.py:789} - INFO - Building test dataloader
[2025-05-03 22:39:32,992]{model.py:297} - INFO - Encoding feature user_id
[2025-05-03 22:39:33,016]{model.py:297} - INFO - Encoding feature item_id
[2025-05-03 22:39:33,041]{model.py:297} - INFO - Encoding feature tag_id


Test epoch #-1:   0%|          | 0/49 [00:00<?, ?it/s]

[2025-05-03 22:39:38,315]{model.py:567} - INFO - Finished Test Epoch #-1, average metrics - [AUC: 0.93801, log_loss: 0.30535]


{'AUC': 0.9380089869168355, 'log_loss': 0.30535252850359723}

# FinalNet

In [19]:
finalnet_model = FinalNet(
    block_type="2B",
    use_field_gate=True,
    use_batch_norm=True,
    add_bias=True,
    block1_hidden_dims=[256, 512, 1024],
    block1_hidden_activations=nn.ReLU,
    block1_dropout_rates=0.2,
    block2_hidden_dims=[256, 512, 1024],
    block2_hidden_activations=nn.ReLU,
    block2_dropout_rates=0.2,
    residual_type="concat",
    proj_output_embeddings=True,
)

In [20]:
train_metrics_final, val_metrics_final = finalnet_model.fit(
    features=df_train.drop(columns="label"),
    target=df_train["label"],
    val_features=df_val.drop(columns="label"),
    val_target=df_val["label"],
    optimizer_cls="torch.optim.Adam",
    optimizer_params=dict(lr=1e-3),
    scheduler_cls="torch.optim.lr_scheduler.ReduceLROnPlateau",
    scheduler_params=dict(mode="max", factor=0.1, patience=2, min_lr=1e-6),
    grad_clip_threshold=10.,
    num_epochs=10,
    seed=42,
    artifacts_path="./finalnet_artifacts",
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
    eval_metric_name="AUC",
    eval_mode="max",
    embedded_features=["user_id", "item_id", "tag_id"],
    oov_masking_proba=0.05,
    default_embedding_size=64, # should be equal for all features if using field gate
    l2_net_reg=0.,
    l2_embedding_reg=0,
)

[2025-05-03 22:50:49,224]{model.py:660} - INFO - Used features config: FeaturesConfig(features=[Feature(name='user_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=64, embedding_vocab_size=16976, embedding_padding_idx=None), Feature(name='item_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=64, embedding_vocab_size=23605, embedding_padding_idx=None), Feature(name='tag_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=64, embedding_vocab_size=49658, embedding_padding_idx=None)])
[2025-05-03 22:50:49,227]{model.py:675} - INFO - Artifacts path is /kaggle/working/finalnet_artifacts
[2025-05-03 22:50:49,227]{model.py:681} - INFO - Best model path is finalnet_artifacts/best_model.pt
[2025-05-03 22:50:49,237]{model.py:686} - INFO - Building features mappings
[2025-05-03 22:50:49,237]{model.py:272} - INFO - Building 

Train epoch #0:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 22:52:05,594]{model.py:567} - INFO - Finished Train Epoch #0, average metrics - [loss: 0.48856]


Validation epoch #0:   0%|          | 0/98 [00:00<?, ?it/s]

[2025-05-03 22:52:14,275]{model.py:567} - INFO - Finished Validation Epoch #0, average metrics - [AUC: 0.88632, log_loss: 0.35258]
[2025-05-03 22:52:24,789]{model.py:747} - INFO - Best model with AUC = 0.8863208422165121 was saved to finalnet_artifacts/best_model.pt


...

[2025-05-03 23:06:20,594]{model.py:567} - INFO - Finished Validation Epoch #8, average metrics - [AUC: 0.94060, log_loss: 0.28852]
[2025-05-03 23:06:31,126]{model.py:747} - INFO - Best model with AUC = 0.9406039642532047 was saved to finalnet_artifacts/best_model.pt


Train epoch #9:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 23:07:46,860]{model.py:567} - INFO - Finished Train Epoch #9, average metrics - [loss: 0.21908]


Validation epoch #9:   0%|          | 0/98 [00:00<?, ?it/s]

[2025-05-03 23:08:05,692]{model.py:567} - INFO - Finished Validation Epoch #9, average metrics - [AUC: 0.94129, log_loss: 0.29179]
[2025-05-03 23:08:16,221]{model.py:747} - INFO - Best model with AUC = 0.9412890242553024 was saved to finalnet_artifacts/best_model.pt
[2025-05-03 23:08:16,222]{model.py:763} - INFO - Loading best model from finalnet_artifacts/best_model.pt


In [21]:
test_metrics_final = finalnet_model.test(
    features=df_test.drop(columns="label"),
    target=df_test["label"],
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
)
test_metrics_final

[2025-05-03 23:08:54,873]{model.py:789} - INFO - Building test dataloader
[2025-05-03 23:08:54,876]{model.py:297} - INFO - Encoding feature user_id
[2025-05-03 23:08:54,903]{model.py:297} - INFO - Encoding feature item_id
[2025-05-03 23:08:54,925]{model.py:297} - INFO - Encoding feature tag_id


Test epoch #-1:   0%|          | 0/49 [00:00<?, ?it/s]

[2025-05-03 23:08:59,488]{model.py:567} - INFO - Finished Test Epoch #-1, average metrics - [AUC: 0.94116, log_loss: 0.29209]


{'AUC': 0.9411648940751596, 'log_loss': 0.29209004007438505}